<a href="https://colab.research.google.com/github/chloe-nguyenminh/Movie_Genre_Detector_Project/blob/main/Copy_of_Movie_Detector_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Movie Detector Final

In [ ]:
# Set up: Importing numpy, pandas, etc.
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from numpy.random import RandomState
from tensorflow.keras import datasets, layers, models, Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# READ DATA
data = pd.read_csv("https://raw.githubusercontent.com/n-d-d/LearnAI-Repo/main/cleaned_data_3.csv")

In [ ]:
import re
genre_set = set()

for i in range(data.shape[0]):
  lst = re.findall(r'\w+', data.Genre[i])
  data['Genre'][i] = lst
  for n in range(len(lst)):
    genre_set.add(lst[n])

data1 = data[["imdbId", "Genre", "Poster"]]
print('genre_set: ', genre_set)
print(data1)

In [ ]:
for genre in genre_set:
  data1[genre] = 0
  for i in range(len(data1)):
    if genre in data1.iloc[i]['Genre']:
      data1.loc[i, genre] = 1

print(data1)

In [ ]:
# Webscrape images from movie posters link

In [ ]:
!apt install chromium-chromedriver

In [ ]:
pip install selenium

In [ ]:
from selenium import webdriver
import requests
import io
from PIL import Image

In [ ]:
def download_image(download_path, url, ID):
  try: 
    image_content = requests.get(url, stream=True).content
    image_file = io.BytesIO(image_content)
    image = Image.open(image_file)
    file_path = download_path + str(ID) + '.jpg'

    with open(file_path, 'wb') as f:
      image.save(f, "JPEG")
  except Exception as e:
    print('Error', e)

In [ ]:
for i in range(1, len(data1)):
   ID = data1['imdbId'][i]
   url = data1['Poster'][i]
   download_image(output_path1, url, ID)

In [ ]:
# Add images in data to folders of respective categories
# One image may belong to multiple genres

In [ ]:
!pip install opencv-python imutils scikit-learn keras tensorflow

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Movie_Poster_Project/Dataset_clean/data_no_dupl_with_exist

In [ ]:
import shutil

# source directory with all imgs
src_dir = '/content/drive/MyDrive/Movie_Poster_Project/Dataset_clean/Posters_from_data3/'

# destination directories
train_dir = '/content/drive/MyDrive/Movie_Poster_Project/Dataset_clean/train/'
test_dir = '/content/drive/MyDrive/Movie_Poster_Project/Dataset_clean/test/'

test_percent = 0.2

# loop through all the images in the source directory
for filename in os.listdir(src_dir):
    if filename.endswith('.jpg'):
        movie_imdb_id = filename.split('/')[-1].replace('.jpg', '')
        print(movie_imdb_id)
        dataframe_row_index = df[df['imdbId'] == int(movie_imdb_id)].index.tolist()[0]
        dataframe_row = df.iloc[dataframe_row_index]
        for genre in genre_set.keys():
          print('test', dataframe_row[genre])
          if df.loc[dataframe_row_index][genre] == 1:
              # move the image to the train or test directory
              if np.random.uniform(0, 1) <= test_percent:
                  shutil.copy(os.path.join(src_dir, filename), os.path.join(test_dir, genre))
              else:
                  shutil.copy(os.path.join(src_dir, filename), os.path.join(train_dir, genre))


In [ ]:
#Read data after processing
df = pd.read_csv('/content/drive/MyDrive/Movie_Poster_Project/Dataset_clean/data_no_dupl_with_existing_posters.csv')

In [ ]:
# resizing
img_width = 253
img_height = 253

image_dir = '/content/drive/MyDrive/Movie_Poster_Project/Dataset_clean/train'
image_dir2 = '/content/drive/MyDrive/Movie_Poster_Project/Dataset_clean/test'

train_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=image_dir,
    image_size=(img_width, img_height),
    shuffle=True,
    batch_size=32,
    label_mode='int'
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    directory=image_dir2,
    image_size=(img_width, img_height),
    shuffle=True,
    batch_size=32,
)

print(train_dataset)


In [ ]:
class_names = train_dataset.class_names

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Added layer of augmentation
data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal", input_shape=(img_height, img_width, 3)), 
                                      layers.RandomRotation(0.1), layers.RandomZoom(0.1)])

In [ ]:
num_classes = len(class_names)
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

epochs=10
history = model.fit(
  train_dataset,
  batch_size=32,
  epochs=epochs,
  validation_data=test_dataset,
)